In [ ]:
from data_collection import extract_content_as_json, fetch_latest_posts

/Users/nayankumarjha/Desktop/WordPressBot/wordpressAIBOT/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/nayankumarjha/Desktop/WordPressBot/wordpressAIBOT/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


: 

In [1]:
orignal_post = fetch_latest_posts(url="https://public-api.wordpress.com/rest/v1.1/sites/portfo336.wordpress.com/posts")

NameError: name 'fetch_latest_posts' is not defined

In [9]:
for post in orignal_post["posts"]:
    print(post["author"]["name"])

Nayan Jha
Nayan Jha


KeyError: 'post'

In [2]:
post = extract_content_as_json(wordpress_domain="portfo336.wordpress.com")

In [3]:
create_faiss_indexes(post_data=post)
vector_store = merge_vector_stores(vector_store_root_dir="doc_embedding_store")

[Document(page_content='Langchain: The Future of Conversational AI in WordPress Blogs'), Document(page_content='Conversational AI has made tremendous strides in recent years, with advancements in natural language processing (NLP) allowing for more human-like interactions with machines. Langchain is one of the prominent frameworks that’s at the forefront of this revolution, enabling developers and content creators to build sophisticated conversational agents. In this essay, we will explore the key features of Langchain, its applications in WordPress blogs, and the benefits it offers to bloggers and'), Document(page_content='to bloggers and website owners.'), Document(page_content='What is Langchain?\nLangchain is a comprehensive framework designed to simplify the development of applications involving language models, such as OpenAI’s GPT-3.5 and similar AI-based models. The framework provides a set of tools and abstractions that facilitate the creation of conversational agents, automate

In [18]:
CONDENSE_QUESTION_PROMPT = """
I want you to act as wordpress post analyzer and based on the provide context which will be the piece of my wordpress post you need to answer the question.
"""

In [92]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
import os
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate
from langchain_community.vectorstores import FAISS
from langchain.memory import VectorStoreRetrieverMemory
from langchain.retrievers.merger_retriever import MergerRetriever
from typing import Tuple

GROQ_API_KEY = "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"

def query_chain(vector_store: MergerRetriever)-> Tuple[ConversationalRetrievalChain, ConversationBufferMemory]:
    chat = ChatGroq(temperature=0, model_name="llama3-8b-8192", api_key=GROQ_API_KEY)
    memory = ConversationBufferMemory(
    memory_key='chat_history', return_messages=True, output_key='answer')
    conversation_chain = (ConversationalRetrievalChain.from_llm
                        (llm=chat,
                        retriever=vector_store,
                        memory=memory,
                        return_source_documents=True,verbose=True))
    print("Conversational Chain created for the LLM using the vector store")
    return conversation_chain,memory

In [93]:
conversation_chain,memory = query_chain(vector_store=vector_store)

Conversational Chain created for the LLM using the vector store


In [95]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [113]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain_core.messages import HumanMessage


def runnable_chain(vector_store):
    llm = ChatGroq(temperature=0, model_name="llama3-8b-8192", api_key=GROQ_API_KEY)

    contextualize_q_system_prompt = """Given a chat history and the latest user question \
    which might reference context in the chat history, formulate a standalone question \
    which can be understood without the chat history. Do NOT answer the question, \
    just reformulate it if needed and otherwise return it as is."""
    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    history_aware_retriever = create_history_aware_retriever(
        llm, vector_store, contextualize_q_prompt
    )
    qa_system_prompt = """You are an assistant for question-answering tasks. \
    Use the following pieces of retrieved context to answer the question. \
    If you don't know the answer, just say that you don't know. \
    Use three sentences maximum and keep the answer concise.\

    {context}"""
    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", qa_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )


    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)
    return rag_chain

In [114]:
rag_chain = runnable_chain(vector_store=vector_store)

In [125]:
chat_history = []

question = "can you tell me about my post on RAG?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history, })
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

second_question = "What is this post about"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

This post is about Retrieval-Augmented Generation (RAG), a new approach that leverages Large Language Models (LLMs) to automate knowledge search, synthesis, extraction, and planning from unstructured data sources.


In [126]:
ai_msg_1

{'input': 'can you tell me about my post on RAG?',
 'chat_history': [HumanMessage(content='can you tell me about my post on RAG?'),
  'I can help you with that! Your post discusses Retrieval-Augmented Generation (RAG), a new approach that leverages Large Language Models (LLMs) to automate knowledge search, synthesis, extraction, and planning from unstructured data sources.'],
 'context': [Document(page_content='End of Post', metadata={'title': 'Langchain: The Future of Conversational AI in WordPress Blogs', 'short_url': 'https://wp.me/pfK043-s', 'date': Timestamp('2024-05-03 14:28:41+0530', tz='UTC+05:30'), 'date_modified': Timestamp('2024-05-03 14:28:41+0530', tz='UTC+05:30')}),
  Document(page_content='Overall, the RAG approach enhances the capabilities of LLMs by enabling them to leverage external knowledge sources in a systematic and efficient manner. This can lead to more powerful and contextually aware applications in various domains, such as natural language understanding, infor